In [269]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import sys
import os
import re
from collections import Counter
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import copy
import pickle
import re
import math

In [258]:
# Download the stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/myasin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [168]:
# Import the csv file with all downloaded READMES
df = pd.read_csv("../user_data_csv/csv_readme_per_user.csv")
df.head(10)

,USERNAMES,READMES
0,alexbw,Sampling Inference for Bayesian HSMMs and HMMs...
1,iassael,DEARanking \n Proposing a hybrid DEA/Polynomia...
2,bshillingford,id3tag-fix codehackathon2014 \n Team members:\...
3,hmansell,NaN
4,clementfarabet,sys \n Has moved to a more community friendly ...
5,ajabri,pytorch-maml \n This is a PyTorch implementati...
6,USERNAMES,READMES
7,coryf,homebrew \n Personal homebrew formula Sudoku C...
8,jrodrigomg,"SERVIDOR NODEJS PROYECTO 1, SISTEMAS OPERATIVO..."
9,vascofernandes,CustomersVRPF \n The project consists of two m...


In [122]:
# Apply different methods to clean the text e.g. remove appestorph and unnessary signs
def clean_df(df):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    df['READMES'] = df['READMES'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
    df["READMES"] = df["READMES"].apply(str)
    df["READMES"] = df["READMES"].str.replace(r"[\"\',]", '')
    df["READMES"] = df["READMES"].apply(lambda x: ''.join(ch for ch in x if ch not in set(symbols)))
    df['READMES'] = df['READMES'].str.replace('\d+', '') 
    df['READMES'] = df['READMES'].str.lower()
    df = df.dropna(how='all')
    return df


In [123]:
# Remove punctiation
def remove_punctuation(df):
    df["READMES"] = df['READMES'].str.replace('[^\w\s]','')
    return df

In [124]:
# Remvoe all stopwords from the READMES column
def remove_stop_words(df):
    # import the stopwords from the nltk library
    stop_words = stopwords.words('english')
    df["READMES"] = df["READMES"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df

In [125]:
# Remove all single characters from the READMES column
def remove_single_characters(df):
    df['READMES'] = df['READMES'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
    return df

In [126]:
# # Stemming the READMES form the dataframe column
# def stemming(df):
#     stemmer= PorterStemmer()
#     df['READMES'] = df['READMES'].apply(lambda x: [stemmer.stem(y) for y in x])
#     return df

In [137]:
# Call all function to clean and preprosses the READMES
def preprocess(df):
    df = clean_df(df)
    df = remove_stop_words(df)
    df = remove_punctuation(df) 
    df = remove_single_characters(df)
    df = df[df['READMES'].notna()]
    # df = stemming(df)
    df.to_csv("cleaned_df.csv", index=False)
    return df
    

In [ ]:
df = df[df['READMES'].notna()]
df.to_csv("cleaned_df.csv", index=False)

In [ ]:
def word_frequency(df):
    word_frequency_dict = {}
    for username,readme in zip(df["USERNAMES"], df["READMES"].str.lower()):
        word_frequency_dict[username] = {}
        for word in readme.split():
            if word not in word_frequency_dict[username].keys():
                word_frequency_dict[username][word] = 1
            else:
                word_frequency_dict[username][word] += 1
        word_frequency_dict[username] = (sorted(word_frequency_dict[username].items(), key=lambda x: -x[1]))
    return word_frequency_dict


In [262]:
df = pd.read_csv("cleaned_df.csv")
df = df[df['READMES'].notna()]
df.head(10)

,USERNAMES,READMES
0,alexbw,sampling inference bayesian hsmms hmms python ...
1,iassael,dearanking proposing hybrid dea polynomial int...
2,bshillingford,idtag fix codehackathon team members brendan s...
4,clementfarabet,sys moved community friendly repo xlua moved c...
5,ajabri,pytorch maml pytorch implementation supervised...
6,USERNAMES,readmes
7,coryf,homebrew personal homebrew formula sudoku curses
8,jrodrigomg,servidor nodejs proyecto sistemas operativos e...
9,vascofernandes,customersvrpf project consists two main parts ...
10,seba-1511,arduino chrono arduino based chronometer toosk...


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(df):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(df['READMES'])]

df['READMES'] = df.apply(lemmatize_text, axis=1)

In [263]:

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.01, stop_words='english')
vectors = vectorizer.fit_transform(df["READMES"])


In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names

In [ ]:
termfreq = vectorizer.vocabulary_
termfreq

In [266]:
query = "Computer vison"

In [267]:
query_vec = vectorizer.transform([query])

In [279]:
results = cosine_similarity(vectors,query_vec).reshape((-1,))

In [280]:
results

array([0.        , 0.03132484, 0.        , 0.00186666, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0212221 , 0.00077672, 0.        , 0.        , 0.        ,
       0.        , 0.01239263, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00301312, 0.        , 0.01311174, 0.        ,
       0.01594991, 0.00964138, 0.        , 0.0096587 , 0.        ,
       0.        , 0.01870219, 0.        , 0.        , 0.        ,
       0.00750794, 0.        , 0.00393149, 0.03187163, 0.0036445 ,
       0.        , 0.01126518, 0.00298295, 0.        , 0.        ,
       0.00056448, 0.        , 0.00821828, 0.        , 0.        ,
       0.00788182, 0.        , 0.        , 0.        , 0.        ,
       0.02371912, 0.        , 0.        , 0.01109768, 0.        ,
       0.        , 0.        , 0.        , 0.0093301 , 0.00682192,
       0.        , 0.        , 0.0013319 , 0.        , 0.01074

In [282]:
# Print Top 10 results
for i in results.argsort()[-10:][::-1]:
    print(df.iloc[i,0])


themadarchitect
edgarriba
intermilan
rAm1n
amirsaffari
mirceamironenco
SalemAmeen
alxndrkalinin
gr33ndata
valthom


In [ ]:
def get_recommendations(df, query):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(vectors,query_vec).reshape((-1,))
    for name in results.argsort()[-10:][::-1]:
        print(df.iloc[name,0])
    return results.argsort()[-10:][::-1]